# Trabajo Practico N°1 NLP
Integrantes: 
1. Rodriguez y Barros, Francisco : 
2. Slepoy, David : S-5782/7

### Importamos las librerias

In [24]:
import spacy
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity

### Cargamos el modelo spacy para usarlo en español

In [25]:
nlp = spacy.load('es_core_news_lg')
# doc = nlp("Estoy muy emocionado por mis vacaciones")
# tokens = [token.text for token in doc]

### Cargamos BERT para las tranformaciones para que capten la semantica

In [26]:
# Cargar un modelo preentrenado como BERT
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

### Creamos el data set con los estados de animo para entrenar el modelo de regresion logistica

In [27]:
data = {
    "frase": [
        # 50 frases para "Alegre"
        "Feliz", "Contento", "Emocionado", "Entusiasmado", "Optimista",
        "Hoy es un día perfecto", "Me siento lleno de energía", "Todo está saliendo bien",
        "Hoy logré todo lo que quería", "El sol brilla y mi ánimo también",
        "Estoy disfrutando cada momento", "Qué día tan increíble", "Me siento agradecido por todo",
        "Todo es tan hermoso hoy", "Me encanta la vida", "Estoy muy feliz de estar aquí",
        "Tengo muchas razones para sonreír", "Siento una paz interior", "Todo es posible hoy",
        "Estoy rodeado de cosas buenas", "Este es un día memorable", "Estoy lleno de vida",
        "Hoy es un gran día para hacer cosas", "Qué maravilloso es vivir", "Siento que puedo lograr todo",
        "La vida es increíble", "Me siento afortunado", "Qué momento tan bonito", "Me siento lleno de amor",
        "Hoy todo es perfecto", "Estoy en paz", "El día no podría ser mejor", "Tengo una gran sonrisa en la cara",
        "Hoy nada puede salir mal", "El mundo está lleno de posibilidades", "Me siento en la cima del mundo",
        "Todo lo que hago hoy me trae alegría", "La vida es muy buena", "Todo lo que toco hoy es éxito",
        "No puedo dejar de sonreír", "Estoy en un estado de felicidad total", "Cada segundo de este día es perfecto",
        "Hoy todo es alegría", "Estoy muy agradecido por lo que tengo", "Todo está fluyendo de manera increíble",
        "Estoy muy emocionado por lo que viene", "Estoy disfrutando del día al máximo",
        "Todo lo que hago me llena de satisfacción", "Estoy extremadamente alegre",
        "La vida es un regalo y hoy lo siento plenamente",
        # 50 frases para "Melancólico"
        "Triste", "Desmotivado", "Deprimido", "Cansado", "Sin ganas",
        "Hoy me siento muy solo", "Todo parece tan difícil", "No veo salida a mis problemas",
        "Hoy no quiero hacer nada", "No tengo energías para seguir", "El día es gris, igual que mi ánimo",
        "Me siento derrotado", "Nada tiene sentido hoy", "Siento que no puedo avanzar",
        "Todo se siente pesado y agotador", "No hay nada que me haga sonreír hoy", "Siento una profunda tristeza",
        "No puedo dejar de pensar en lo mal que va todo", "Hoy no encuentro la luz en nada",
        "Todo parece ser en vano", "Hoy no tengo fuerzas para seguir", "Me siento atrapado",
        "Nada va bien en mi vida", "Me siento sin rumbo", "No tengo esperanza en nada",
        "Me siento desconectado de todo", "Todo me agobia", "Me siento solo en este mundo",
        "No sé cómo salir de este estado", "El futuro se ve oscuro", "Estoy roto por dentro",
        "No puedo dejar de sentirme mal", "Estoy en un pozo del que no puedo salir",
        "Hoy es un día sin sentido", "No encuentro motivos para seguir", "Todo parece una carga insostenible",
        "Me siento completamente vacío", "Nada me motiva hoy", "Siento una tristeza que no se va",
        "No hay nada que me anime", "Todo está mal hoy", "Me siento totalmente perdido",
        "No tengo fuerzas para seguir adelante", "Hoy todo es oscuridad", "Siento que todo está fuera de control",
        "No tengo ganas de nada", "Todo parece ser una carga", "Hoy no veo salida a mis problemas",
        "No encuentro nada positivo", "Nada tiene sentido en este día",
        # 50 frases para "Ni fu ni fa"
        "Indiferente", "Neutro", "Normal", "Regular", "Nada especial",
        "Hoy no siento nada en particular", "No es un mal día, pero tampoco es bueno",
        "Hoy no pasa nada interesante", "Me siento igual que siempre", "Nada nuevo hoy",
        "El día está bien, nada especial", "No me quejo, pero tampoco estoy feliz",
        "Hoy es un día como cualquier otro", "No me siento ni bien ni mal", "Todo sigue su curso normal",
        "Es solo otro día común", "No tengo grandes emociones hoy", "Hoy no tengo sentimientos fuertes",
        "El día es tranquilo, sin sobresaltos", "No tengo nada relevante que contar",
        "Hoy todo parece igual que siempre", "El día está pasando sin nada memorable",
        "Hoy es un día sin altibajos", "No ha pasado nada fuera de lo normal",
        "Me siento completamente neutro hoy", "Nada interesante ha sucedido", "Hoy es solo un día más",
        "No hay mucho que decir del día", "Todo está bien, sin más", "Hoy no ha pasado nada importante",
        "Hoy es un día sin grandes cambios", "Es un día tranquilo", "Todo parece estar en su lugar, nada más",
        "El día está bien, sin emociones fuertes", "No siento nada especial hoy",
        "Hoy me siento en un estado neutro", "No tengo ninguna emoción intensa",
        "Hoy es un día equilibrado, nada más", "Es un día más, sin grandes cosas que contar",
        "Todo ha estado normal hoy", "El día ha sido bastante tranquilo", "Hoy no ha sido ni bueno ni malo",
        "No siento nada fuerte hoy", "Es un día sin sobresaltos", "Hoy todo ha sido bastante regular",
        "No hay mucho que comentar del día", "Es un día como cualquier otro", "No ha pasado nada relevante hoy",
        "Hoy todo ha sido bastante normal", "Me siento en un estado intermedio"
    ],
    "estado_animo": [
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa"
    ]
}

# Convertir a DataFrame
df = pd.DataFrame(data)

### Transformamos las frases de entrenamiento del dataset con el transform de BERT, luego hacemos label encoder para transformar las etiquetas de los estados de animos en numericas 

In [28]:
embeddings_train = model.encode(df["frase"].tolist())

# Convertir las etiquetas de texto a numericas
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df["estado_animo"])

# Entrenar el modelo con Regresión Logística
clf = LogisticRegression(random_state=42, max_iter=200)
clf.fit(embeddings_train, y_train)

LogisticRegression(max_iter=200, random_state=42)

### Ingresa la frase el ususario, lo transformamos con BERT, y predecimos a que estado de animo corresponde lo ingresado

In [29]:
# Solicitar input del usuario
frase_usuario_sentimiento = input("Ingresa una frase para clasificar tu estado de ánimo: ")

# Convertir la frase del usuario en embeddings
embedding_frase_usuario_sentimiento = model.encode([frase_usuario_sentimiento])

# Predecir el estado de ánimo
estado_animo_predicho = clf.predict(embedding_frase_usuario_sentimiento)

# Convertir la predicción numérica de vuelta a texto
estado_animo_predicho_texto = label_encoder.inverse_transform(estado_animo_predicho)
print(f"El estado de ánimo predicho es: {estado_animo_predicho_texto[0]}")

frase_usuario_tematica = input("Ingresa una temática que le gustaría explorar: ")

total = frase_usuario_sentimiento + ' ' + estado_animo_predicho_texto[0] + ' ' + frase_usuario_tematica

El estado de ánimo predicho es: Alegre


In [30]:
print(total)

Alegre Alegre Dragones


# Embeding de datasets

Carga de los datasets

In [31]:
# Dataset de juegos de mesa
df_juegos_de_mesa = pd.read_csv('bgg_database.csv')
# Dataset de películas
df_peliculas = pd.read_csv('IMDB-Movie-Data.csv')
# Dataset de películas
df_libros = pd.read_csv('df_libros.csv')

Preprocesamiento de los textos en los tres datasets para dejar los textos listos para la generación de embeddings.

El preprocesamiento incluirá:

* Tokenización.
* Eliminación de stopwords.
* Lematización (opcional).

## Unificar las columnas preprocesadas

In [32]:
df_peliculas['full_text'] = df_peliculas['Genre'] + ' ' + df_peliculas['Description']
df_libros['full_text'] = df_libros['Subjects'] + ' ' + df_libros['Summary']
df_juegos_de_mesa['full_text'] = df_juegos_de_mesa['categories'] + ' ' + df_juegos_de_mesa['description']

### Generar embeddings a partir de los textos unificados

In [33]:
# # Para películas
# peliculas_embeddings = model.encode(df_peliculas['full_text'].tolist())
# df_peliculas['embeddings'] = peliculas_embeddings.tolist()

# # Para libros
# libros_embeddings = model.encode(df_libros['full_text'].tolist())
# df_libros['embeddings'] = libros_embeddings.tolist()

# Para juegos
# juegos_embeddings = model.encode(df_juegos_de_mesa['full_text'].tolist())
# df_juegos_de_mesa['embeddings'] = juegos_embeddings.tolist()

### Transformamos en csv a los embessing de cada elemento en peliculas, libros y juegos

In [34]:
# df_peliculas.to_csv('df_peliculas_emb.csv', index=False)
# df_libros.to_csv('df_libros_emb.csv', index=False)
# df_juegos_de_mesa.to_csv('df_juegos_de_mesa_emb.csv', index = False)

In [35]:
# Dataset de juegos de mesa
df_juegos_de_mesa_1 = pd.read_csv('df_juegos_de_mesa_emb.csv')
# Dataset de películas
df_peliculas_1 = pd.read_csv('df_peliculas_emb.csv')
# Dataset de películas
df_libros_1 = pd.read_csv('df_libros_emb.csv')

In [54]:
df_libros_1.head()

,Title,Author,Summary,Subjects,Release Date,Language,Url,full_text,embeddings
0,frankenstein modern prometheus,"Shelley, Mary Wollstonecraft, 1797-1851",frankenstein modern prometheus mary wollstonec...,science fiction horror tale gothic fiction sci...,"Oct 1, 1993",English,https://www.gutenberg.org/ebooks/84,frankenstein modern prometheus science fiction...,"[0.013650155626237392, 0.06411924213171005, -0..."
1,romeo juliet,"Shakespeare, William, 1564-1616",romeo juliet william shakespeare tragedy likel...,vendetta drama youth drama verona italy drama ...,"Nov 1, 1998",English,https://www.gutenberg.org/ebooks/1513,romeo juliet vendetta drama youth drama verona...,"[0.016256380826234818, 0.007097302470356226, 0..."
2,moby dick whale,"Melville, Herman, 1819-1891",moby dick whale herman melville novel write mi...,whaling fiction sea story psychological fictio...,"Jul 1, 2001",English,https://www.gutenberg.org/ebooks/2701,moby dick whale whaling fiction sea story psyc...,"[0.014827858656644821, 0.07577294111251831, -0..."
3,pride prejudice,"Austen, Jane, 1775-1817",pride prejudice jane austen classic novel writ...,england fiction young woman fiction love story...,"Jun 1, 1998",English,https://www.gutenberg.org/ebooks/1342,pride prejudice england fiction young woman fi...,"[-0.026456192135810852, 0.03377552703022957, -..."
4,middlemarch,"Eliot, George, 1819-1880",middlemarch george eliot novel write mid-19th ...,didactic fiction city town life fiction englan...,"Jul 1, 1994",English,https://www.gutenberg.org/ebooks/145,middlemarch didactic fiction city town life fi...,"[-0.019195305183529854, 0.0279136560857296, 0...."


In [47]:

def recomendar(frase1_mas_estado_mas_frase2):
    # Vectorizamos la frase
    vector_comparar = model.encode(frase1_mas_estado_mas_frase2)

    # Obtener los embeddings de cada dataset
    juegos_ubicacion = df_juegos_de_mesa_1['embeddings'].tolist()
    peliculas_ubicacion = df_peliculas_1['embeddings'].tolist()
    libros_ubicacion = df_libros_1['embeddings'].tolist()

    # Calcular similitudes
    similitudes_juegos = cosine_similarity([vector_comparar], juegos_ubicacion)
    similitudes_peliculas = cosine_similarity([vector_comparar], peliculas_ubicacion)
    similitudes_libros = cosine_similarity([vector_comparar], libros_ubicacion)

    # Obtener los índices de las top 3 recomendaciones
    top_indices_juegos = similitudes_juegos.argsort()[0][-3:][::-1]  # Top 3 juegos
    top_indices_peliculas = similitudes_peliculas.argsort()[0][-3:][::-1]  # Top 3 películas
    top_indices_libros = similitudes_libros.argsort()[0][-3:][::-1]  # Top 3 libros

    # Crear diccionarios con las recomendaciones
    recomendaciones_juegos = [
        {
            'titulo': df_juegos_de_mesa_1.iloc[index]['titulo'],
            'summary': df_juegos_de_mesa_1.iloc[index]['summary']
        } for index in top_indices_juegos
    ]

    recomendaciones_peliculas = [
        {
            'titulo': df_peliculas_1.iloc[index]['titulo'],
            'summary': df_peliculas_1.iloc[index]['summary']
        } for index in top_indices_peliculas
    ]

    recomendaciones_libros = [
        {
            'titulo': df_libros_1.iloc[index]['titulo'],
            'summary': df_libros_1.iloc[index]['summary']
        } for index in top_indices_libros
    ]

    # Devolver un diccionario con las recomendaciones
    return {
        'juegos': recomendaciones_juegos,
        'peliculas': recomendaciones_peliculas,
        'libros': recomendaciones_libros
    }

In [48]:
recomendar(total)

ValueError: could not convert string to float: np.str_('[-0.04938803240656853, -0.01134925801306963, -0.02097523771226406, 0.027507206425070763, -0.042637743055820465, 0.0075018987990915775, 0.004868983756750822, 0.01760764792561531, 0.02213479019701481, 0.03568726405501366, 0.005124249961227179, 0.06333374977111816, 0.044197652488946915, 0.035748809576034546, 0.004900821018964052, 0.03630192577838898, -0.017650678753852844, 0.02889787219464779, -0.04262211173772812, 0.0024098032154142857, -0.021424349397420883, -0.0013152419123798609, -0.016024161130189896, -0.0096848513931036, 0.021618954837322235, 0.035320479422807693, 0.012545173987746239, 0.010263212956488132, -0.003823239356279373, -0.040526483207941055, 0.049252476543188095, -0.04624750465154648, -0.005751725751906633, 0.03282445669174194, 2.056216771961772e-06, -0.04706258326768875, -0.021539298817515373, -0.026890244334936142, 0.03341910243034363, -0.0005627389182336628, 0.06549785286188126, 0.008288262411952019, 0.01490539778